<a href="https://colab.research.google.com/github/diegoestradaXO/Lab6-Data-Science/blob/master/Lab6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio 6: Analítica de redes sociales
## Autores
- Saúl Contreras 18409
- Andrea Elías 17048
- Diego Estrada 18540

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:
import tweepy
import csv
import pandas as pd

---
## Prueba del API de Twitter con llaves de perfil de desarrollador


In [5]:
auth = tweepy.OAuthHandler('fxRDOaD9DYKj62uAwczglNevG', '0WSdq2MMgltglKb7eMxD7IyJGhT0POh9o4IuqUhPjPRLCYCYA5')
auth.set_access_token('3171113016-1zypbezJcLEyQlM5x9P05CI9EtvSwZYueIBgTH2', '6Q3H0DSqqODFvRFK5HQpDhxZWoaSV1X4P1t2rCRBxB6oG')

api = tweepy.API(auth, wait_on_rate_limit=True)

public_tweets = api.home_timeline()
for tweet in public_tweets:
    print (tweet.text)


Nutella, anticonceptivos y ardillas: el error del siglo XIX que los ingleses siguen intentando solucionar en la act… https://t.co/sQfzNhWZ5S
Buenos diaaaaaas! Feliz domingo! Me hago pis 🙃 

https://t.co/iSNSTT4rdk
Conozca su horóscopo del día, según su signo zodiacal: Aries, Tauro, Géminis, Cáncer, Leo, Virgo, Libra, Escorpio,… https://t.co/UNbnwt7u4O
have you tried #地球グミ Earth Gummies? 🌏 share thisしなこ Shinako / ASMR video if you have. https://t.co/XnjlEX5rqT
#EUCoronavirus El Ministerio de Salud Pública y Asistencia Social actualizó este sábado, 4 de septiembre, el Semáfo… https://t.co/Y1euhAOqI6
#PorSiNoLoVio | El mes pasado fue el mes de junio más caluroso de la historia en Norteamérica, en el cual se batier… https://t.co/UgdjYHh1RK
#PorSiNoLoVio | En la mayoría de los casos, los pies hinchados aparecen como consecuencia de llevar un estilo de vi… https://t.co/tQQHRuRWAg
#PorSiNoLoVio | Cuando se habla del cuidado de las células de la piel, pocos productos de origen natural pueden i

- - -
## Descripción del problema
_Extraiga los datos relacionados con el coviden Guatemala, puede utilizar cualquier red social, a la que  se  tenga  acceso,  y  pueden  extraerse  datos  de  varias.  Por  ejemplo  si  extrae  datos  de  twitter podría utilizar el hashtag #COVID19gt o #CoronavirusGT. Explore los datos que extraiga y descubra conocimiento, tendencias y elementos interesantes._



### Extracción de datos

In [14]:
#Se crea el archivo csv, si ya existe, se escribe en él.
file = open('twitter_covid_gt.csv', 'a')

writer = csv.writer(file)
hashtags = ('#COVID19gt OR #CoronavirusGT OR #CuarentenaTotal OR #GuatemalaNoSeDetiene OR #Guatemala#COVID19 OR #CoronavirusGuatemala OR #QuedateEnCasaGT OR #COVID_19gt OR #Covid19gt OR #ToqueDeQuedaGT OR #PandemiaGT OR #DatosCovidGT OR #YoMeQuedoEnMiCasa OR #JuntosSaldremosAdelante')
for tweet in tweepy.Cursor(api.user_timeline,id='GuatemalaGob', lang="es").items():
    temporal = tweet.text
    #Limpieza de caracteres especiales
    temporal = temporal.replace('á', 'a')
    temporal = temporal.replace('é', 'e')
    temporal = temporal.replace('í', 'i')
    temporal = temporal.replace('ó', 'o')
    temporal = temporal.replace('ú', 'u')
    temporal = temporal.replace('Á', 'A')
    temporal = temporal.replace('É', 'E')
    temporal = temporal.replace('Í', 'I')
    temporal = temporal.replace('Ó', 'O')
    temporal = temporal.replace('Ú', 'U')
    temporal = temporal.replace('ñ', 'n')
    temporal = temporal.replace('Ñ', 'N')
    temporal = temporal.replace('Ü', 'U')
    temporal = temporal.replace('ü', 'u')
    writer.writerow([tweet.created_at, temporal.encode('utf-8')])

### Carga de datos

In [17]:
data = pd.read_csv('/content/twitter_covid_gt.csv')

### Limpieza y preprocesamiento

In [19]:
def remove_characters(text):
  return text.translate(text.maketrans('', '', string.punctuation))

def remove_url(text):
    return re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

def remove_num(text):
    return re.sub('^\d+\s|\s\d+\s|\s\d+$','',text)

Insalación de librería emoji

In [20]:
!pip install emoji

     |████████████████████████████████| 184 kB 3.2 MB/s 
  Created wheel for emoji: filename=emoji-1.4.2-py3-none-any.whl size=186469 sha256=548598e3b5d07fd499ecc41f7c273758d1c6b525669fd2cc1e965b4ec4da6a1f
  Stored in directory: /root/.cache/pip/wheels/e4/61/e7/2fc1ac8f306848fc66c6c013ab511f0a39ef4b1825b11363b2
Successfully built emoji
